In [ ]:
# courant criteria
import numpy as np
1/np.sqrt(3*(1/6e-3)**2)/(299792458)*1e9

In [ ]:
#import test
import importlib
reload(test)

In [ ]:
%%writefile test.py
import pyfdtd3d.structuredpoints_cupy as fdtd
import pyfdtd3d.structuredpoints_ui as ui
from pyfdtd3d.base import sin, vsrc
import time

class MySolver(ui.Solver):
    def prepare(self):
        p = self.parameters
        self.time = t = fdtd.TimeSeries(p['dt']) # [second]
        self.grid = g = fdtd.Grid(
            dimensions=p['dimensions'], # [cells]
            spacing=p['spacing'], # [mm]
        )
        self.yee = y = fdtd.Yee(grid=g)
        self.material = m = fdtd.Material(*g.dimensions)
        self.pml = fdtd.PMLSolver(yee=y, time=t, thickness=8)
        self.hcalc = fdtd.HCalc(yee=y, time=t)
        self.ecalc = fdtd.ECalc(yee=y, time=t, material=m)
        self.currentz = fdtd.CurrentZ(yee=y)
    def calculate(self):
        y, t, hc, ec, cz = self.yee, self.time, self.hcalc, self.ecalc, self.currentz
        b = self.pml
        dx, dy, dz = y.grid.spacing
        t.progress()
        hc.calculate()
        b.h_calc.dispatch()
        b.h_com.dispatch()
        t.progress()
        ec.calculate()
        b.e_calc.dispatch()
        b.e_com.dispatch()
        y.ez[15, 200, 201:207] = 0.
        y.ez[15, 200, 194:200] = 0.
        y.ez[15, 200, 200] = vsrc(voltage=sin(t.current_time, freq=2e+9), current=cz[15, 200, 200], delta=dz)

s = MySolver(
    dt=0.01e-9, # [second]
    dimensions=(30, 400, 401), # [cells]
    spacing=(6e-3, 6e-3, 6e-3), # [mm]
)
s.prepare()

p = ui.Plotter(
    yee=s.yee,
    slices=(15, slice(None), slice(None)),
    mapper=ui.LinearColorMapper(palette='Viridis256', low=-1, high=1),
)

u = ui.ControlPanel(p, s, counter_limit=1001, plot_every=10)